In [1]:
import bitboards
import autoencoder
import pgn_reader as reader
import torch
import torch.nn as nn
import torch.optim as optim
import random
import math
import gc


gc.collect()
torch.cuda.empty_cache()

dsetSize, batch = 2**14, 2**10
#autoEncoder = autoencoder.autoencoder().cuda()
autoEncoder =  torch.load('autoencoderxallw2.pt').cuda()
criterion = nn.BCELoss(reduction='sum')
optimizer = optim.Adam(autoEncoder.parameters())

class chessDataset(torch.utils.data.Dataset):
    def __init__(self, data_list):
        self.data = data_list
        
    def __getitem__(self, idx):
            return self.data[idx]
    
    def __len__(self):
        return len(self.data)


data = reader.get_dataset(dsetSize)
while data:
    random.shuffle(data)
    
    dataset = chessDataset(data[:math.floor(1/16*len(data))])
    testset = chessDataset(data[math.floor(1/16*len(data)):-1])

    DataLoader = torch.utils.data.DataLoader(dataset, batch_size = batch, shuffle = True)
    TestLoader = torch.utils.data.DataLoader(testset, batch_size = batch, shuffle = True)   
    
    autoEncoder.eval()
    test_loss = 0
    for x in TestLoader:
        x = x.cuda()
        y = autoEncoder(x).cuda()
        loss = criterion(y, x)
        test_loss += loss.item()
    print("Test loss: ", test_loss / len(testset))
        
    autoEncoder.train()
    training_loss = 0
    for x in DataLoader:
        optimizer.zero_grad()
        x = x.cuda()
        y = autoEncoder(x).cuda()
        loss = criterion(y, x)
        loss.backward()
        optimizer.step()
        training_loss += loss.item()
    print("Training loss: ", training_loss / len(dataset), end='\n\n')
    
    data = reader.get_dataset(dsetSize)
    
    torch.save(autoEncoder, 'autoencoderxallw3.pt')

Test loss:  3.50127676693022
Training loss:  3.3617477416992188

Test loss:  16.100759756900047
Training loss:  16.207481803189676

Test loss:  16.920089838780175
Training loss:  16.910531716695647



KeyboardInterrupt: 